In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('./data/train.csv', index_col=0)

print(train.shape)
train.head()

(472972, 2)


,level,full_log
id,,
0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...


In [3]:
train['level'].value_counts()

0    334065
1    132517
3      4141
5      2219
2        12
4        10
6         8
Name: level, dtype: int64

# 전처리

- YEAR, MON, DAY, TIME
- IP
    - 127.0.0.1 -> localhost
    - IP
    - IP.PORT
- HEX 처리 ?
- NUM
- 특수문자 처리

In [4]:
train['first_word'] = train['full_log'].str.extract('^([^ ]+)(?: |$)')

In [5]:
def show_contains(target, *, col='full_log', first_words=None, cnt=None):
    condition = train[col].str.contains(target)
    if first_words is not None:
        condition &= train['first_word'].isin(first_words)
    
    logs = train[condition][col][:cnt] if cnt is not None else train[condition][col]
    for log in logs:
        print(log, '\n')

In [6]:
train[train['full_log'].str.contains('\d{2}:\d{2}:\d{2}')]['first_word'].value_counts()

Jan             207025
Feb              35544
Dec              25882
Oct              23883
Sep              23193
Nov              21875
Mar              12174
type=SYSCALL       152
File                72
2021                22
2020                 9
Name: first_word, dtype: int64

In [7]:
train[train['full_log'].str.contains('\d{4}-\d{2}-\d{2}')]['first_word'].value_counts()

Jan             171098
Dec              23428
Sep              22389
Feb              20855
Oct              14669
Nov               3722
Mar               3285
type=SYSCALL       152
Name: first_word, dtype: int64

In [8]:
show_contains('\d{4}-\d{2}-\d{2}', first_words={'type=SYSCALL'}, cnt=1)

type=SYSCALL msg=audit(1603193109.472:183338): arch=c000003e syscall=2 success=yes exit=3 a0=44dc53 a1=0 a2=1b6 a3=7ffc2cf16ca0 items=1 ppid=29260 pid=29262 auid=4294967295 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=4294967295 comm="mailx" exe="/usr/bin/mailx" subj=system_u:system_r:abrt_t:s0-s0:c0.c1023 key="audit-wazuh-r" type=CWD msg=audit(1603193109.472:183338):  cwd="/var/spool/abrt/oops-2020-10-20-20:25:02-29133-0" type=PATH msg=audit(1603193109.472:183338): item=0 name="/etc/mail.rc" inode=67518562 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:etc_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603193109.472:183338): proctitle=2F62696E2F6D61696C78002D73005B616272745D206B65726E656C3A205741524E494E473A204350553A2031205049443A2030206174206E65742F73636865642F7363685F67656E657269632E633A333030206465765F7761746368646F672B30783234322F3078323530002D720075736572406C6F63616C686F737400726F 



## 날짜, 시간

`Sep 24 10:02:22`
`2020 Oct 30 08:39:31`
`Fri Sep 18 13:36:11 2020`

`2021-02-08T16:21:00` `2021-01-12T07:22:32Z`

In [9]:
train['masked'] = train['full_log'].str.replace('\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z?', '<TS>', regex=True)
train[train['masked'].str.contains('<TS>')]['first_word'].value_counts()

Jan    171098
Dec     23428
Sep     22389
Feb     20855
Oct     14669
Nov      3722
Mar      3285
Name: first_word, dtype: int64

In [10]:
train['masked'] = train['masked'].str.replace('\d{4}(?= (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))', '<YEAR>', regex=True)
train[train['first_word']=='2020']

,level,full_log,first_word,masked
id,,,,
63675,3,2020 Oct 30 08:39:31 WinEvtLog: System: ERROR(...,2020,<YEAR> Oct 30 08:39:31 WinEvtLog: System: ERRO...
226155,3,2020 Oct 28 13:32:06 WinEvtLog: System: ERROR(...,2020,<YEAR> Oct 28 13:32:06 WinEvtLog: System: ERRO...
246839,3,2020 Oct 30 09:55:55 WinEvtLog: System: ERROR(...,2020,<YEAR> Oct 30 09:55:55 WinEvtLog: System: ERRO...
275279,3,2020 Oct 20 09:58:04 WinEvtLog: System: ERROR(...,2020,<YEAR> Oct 20 09:58:04 WinEvtLog: System: ERRO...
279151,3,2020 Oct 19 10:32:47 WinEvtLog: Application: E...,2020,<YEAR> Oct 19 10:32:47 WinEvtLog: Application:...
332210,3,2020 Oct 21 17:59:46 WinEvtLog: System: ERROR(...,2020,<YEAR> Oct 21 17:59:46 WinEvtLog: System: ERRO...
361699,3,2020 Oct 23 08:22:32 WinEvtLog: System: ERROR(...,2020,<YEAR> Oct 23 08:22:32 WinEvtLog: System: ERRO...
393148,3,2020 Oct 22 09:59:04 WinEvtLog: System: ERROR(...,2020,<YEAR> Oct 22 09:59:04 WinEvtLog: System: ERRO...
454020,3,2020 Oct 23 08:23:30 WinEvtLog: System: ERROR(...,2020,<YEAR> Oct 23 08:23:30 WinEvtLog: System: ERRO...


In [11]:
train['masked'] = train['masked'].str.replace(
    '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}\s+\d{2}:\d{2}:\d{2}', '<DATE> <TIME>', regex=True)
train[train['masked'].str.contains('<DATE>')]['first_word'].value_counts()

Jan     207025
Feb      35544
Dec      25882
Oct      23883
Sep      23193
Nov      21875
Mar      12174
File        72
2021        22
2020         9
Name: first_word, dtype: int64

In [12]:
train['masked'] = train['masked'].str.replace(
    '[A-Za-z]{3}\s+<DATE> <TIME>\s+\d{4}', '<DAY> <DATE> <TIME> <YEAR>', regex=True)
train[train['masked'].str.contains('<DAY>')]['first_word'].value_counts()

Jan     2244
File      72
Name: first_word, dtype: int64

## IP

`127.0.0.1` -> localhost

- IP
- IP.PORT

In [13]:
train['masked'] = train['masked'].str.replace('127.0.0.1', 'localhost', regex=True)

print(train[train['full_log'].str.contains('127.0.0.1')]['full_log'].iloc[0], '\n')
print(train[train['full_log'].str.contains('127.0.0.1')]['masked'].iloc[0], '\n')

Jan 19 09:19:14 localhost logstash: [2021-01-19T09:19:14,483][WARN ][logstash.outputs.elasticsearch] Attempted to resurrect connection to dead ES instance, but got an error. {:url=>"http://127.0.0.1:9200/", :error_type=>LogStash::Outputs::ElasticSearch::HttpClient::Pool::HostUnreachableError, :error=>"Elasticsearch Unreachable: [http://127.0.0.1:9200/][Manticore::SocketException] 연결이 거부됨 (Connection refused)"} 

<DATE> <TIME> localhost logstash: [<TS>,483][WARN ][logstash.outputs.elasticsearch] Attempted to resurrect connection to dead ES instance, but got an error. {:url=>"http://localhost:9200/", :error_type=>LogStash::Outputs::ElasticSearch::HttpClient::Pool::HostUnreachableError, :error=>"Elasticsearch Unreachable: [http://localhost:9200/][Manticore::SocketException] 연결이 거부됨 (Connection refused)"} 



In [14]:
train['masked'] = train['masked'].str.replace('\d+\.\d+\.\d+\.\d+(?:\.\d+)?', '<IP>', regex=True)
train[train['masked'].str.contains('<IP>')]['first_word'].value_counts()

Nov             9761
Feb             7082
Oct             4203
Mar             1984
Jan             1319
ossec:          1294
Dec              733
oscap:           286
Sep               95
2020               8
2021               4
File               4
type=SYSCALL       1
Name: first_word, dtype: int64

## HEX, NUM

In [15]:
train['masked'] = train['masked'].str.replace('(?<![0-9a-fA-F])0x[0-9a-fA-F]+(?=\W|$)', '<HEX>', regex=True)
show_contains('<HEX>', col='masked', cnt=5)

<DATE> <TIME> localhost esild-ml-start.sh: , filename = '/var/esild/ml/model1', file descriptor = 3, errno = 21, error message = 'Is a directory', buf = <HEX>, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0) 

<DATE> <TIME> localhost esild-ml-start.sh: , filename = '/var/esild/ml/model1', file descriptor = 3, errno = 21, error message = 'Is a directory', buf = <HEX>, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0) 

<DATE> <TIME> localhost esild-ml-start.sh: , filename = '/var/esild/ml/model1', file descriptor = 3, errno = 21, error message = 'Is a directory', buf = <HEX>, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0) 

<DATE> <TIME> localhost esild-ml-start.sh: , filename = '/var/esild/ml/model1', file descriptor = 3, errno = 21, error message = 'Is a directory', buf = <HEX>, total read size = 8, bytes this sub-read = 8, by

In [16]:
PATTERN_SN = '(?<=\W)(?=[a-fA-F0-9\-]*[0-9])(?=[a-fA-F0-9\-]*[a-fA-F])[a-fA-F0-9]{3,}(?:-[a-fA-F0-9]{3,})+(?=\W|$)'

train['masked'] = train['masked'].str.replace(PATTERN_SN, '<SN>', regex=True)
show_contains('<SN>', col='masked', cnt=5)

File 'HKEY_LOCAL_MACHINE\System\CurrentControlSet\Services\BTHPORT\Parameters\Services\{<SN>}' was added.
 

<DATE> <TIME> localhost kibana: {"type":"log","@timestamp":"<TS>","tags":["warning","monitoring-ui","kibana-monitoring"],"pid":7412,"message":"[export_exception] failed to flush export bulks :: {\"path\":\"/_xpack/monitoring/_bulk\",\"query\":{\"system_id\":\"kibana\",\"system_api_version\":\"6\",\"interval\":\"10000ms\"},\"body\":\"{\\\"index\\\":{\\\"_type\\\":\\\"kibana_stats\\\"}}\\n{\\\"kibana\\\":{\\\"uuid\\\":\\\"<SN>\\\",\\\"name\\\":\\\"localhost.localdomain\\\",\\\"index\\\":\\\".kibana\\\",\\\"host\\\":\\\"<IP>\\\",\\\"transport_address\\\":\\\"<IP>:5601\\\",\\\"version\\\":\\\"6.7.1\\\",\\\"snapshot\\\":false,\\\"status\\\":\\\"green\\\"},\\\"usage\\\":{\\\"kql\\\":{\\\"optInCount\\\":2,\\\"optOutCount\\\":1,\\\"defaultQueryLanguage\\\":\\\"default-lucene\\\"},\\\"localization\\\":{\\\"locale\\\":\\\"ko-KR\\\",\\\"integrities\\\":{\\\"/usr/share/kibana/node_modules/x

In [17]:
train['masked'].str.extractall('(=(?=[a-fA-F0-9]*[a-fA-F])[a-fA-F0-9]+)(?=\W|$)').value_counts()

=fd                  132681
=c000003e            116838
=1b6                  43938
=7f89fd9cc630         10044
=7f89fe522280          7397
                      ...  
=7f42e38d40c0             1
=7f42e33df4f8             1
=7f42e32193a8             1
=7f42e31d95a4             1
=ffffffffffffffff         1
Length: 96991, dtype: int64

In [18]:
train['masked'].str.extractall('(=(?=[a-fA-F0-9]*[a-fA-F])[a-fA-F0-9]{,3})(?=\W|$)').value_counts()

=fd     132681
=1b6     43938
=b64       347
=b32       322
=1c        177
=a         165
=5f         38
=6e         34
=2d         33
=f          21
=1f         21
=b          19
=6b         19
=fff        17
=6c         16
=1a0         9
=6f          6
=2e          6
=c1          4
=7a          3
=1e          2
=c2          2
=1a4         2
=A           1
=C           1
=D           1
=6d          1
=1d          1
=c4          1
=d           1
=1b          1
=ff          1
=1a          1
dtype: int64

In [19]:
train['masked'].str.extractall('(:(?=[a-fA-F0-9]*[a-fA-F])[a-fA-F0-9]+)(?=\s|$)').value_counts()

:7f5d8db18050    1
:7f654f0dae51    1
:7fab61e2ce51    1
:7fee924d8050    1
:7ffd06665c70    1
:7ffdf46f5180    1
:7ffe69ca7b98    1
:7ffe7fc1f438    1
dtype: int64

In [20]:
train['masked'].str.extractall('((?P<first>\'|")[a-fA-F0-9]+(?P=first))')[0].value_counts()

"60"                                          537
"df"                                          258
"1"                                           138
"20"                                          109
"4"                                            96
                                             ... 
'784ce8d8ec6b3e2fd62da5eaf0b360da6457c7e7'      1
"909"                                           1
'd9ffbed68236e39282d79dc92deed96d86f3d8e5'      1
'd83d3323f43744e8726fb493256f1baf'              1
'e7a79804d8b4e25f94b9e9cdaa68d144d2d5de99'      1
Name: 0, Length: 2546, dtype: int64

In [21]:
train['masked'].str.extractall('((?P<first>\'|")[a-fA-F]+(?P=first))')[0].value_counts()

"df"    258
"c"       6
Name: 0, dtype: int64

In [22]:
train['masked'].str.extractall('(\'[a-fA-F0-9]+\')').value_counts()

'503'                                         20
'f27bd8e798c9ac36d085e24f7cdc5c17745ced30'     4
'1fd0dc10ef50bbf663fb64c15e747561'             4
'779ad9e81573218c15f475dd143e7263'             3
'be7e2251682fce605f506c19f616579c'             3
                                              ..
'58354081840c7137c8b900cab5aa1d7947a2d6fb'     1
'5824bad6a9e68c012f03b5242137235a9aa3d594'     1
'57cb3bade57d71f20d9df24bd62b3d9f'             1
'57c1595499c5f2dbd5f03de96fb1a93af35cd2bc'     1
'85c4ffd844c6a0e557a1fa5bd0c675368d5d9d77'     1
Length: 1525, dtype: int64

In [23]:
train['masked'].str.extractall('(\'[0-9]+\')').value_counts()

'503'          20
'1440'          2
'406144052'     2
'232'           2
'846'           2
               ..
'153'           1
'152'           1
'151'           1
'150'           1
'87332'         1
Length: 78, dtype: int64

In [24]:
train['masked'].str.extractall('( (?=[a-fA-F0-9]*[a-fA-F])(?=[a-fA-F0-9]*[0-9])[a-fA-F0-9]+(?: |$))').value_counts()

 db5                  324
 BDB0113              294
 BDB1507              294
 BDB0087              279
 b32                   72
                     ... 
 ffff9428a745e210       1
 ffff9428a745e390       1
 ffff9428a745e8a0       1
 ffff947d7ce77210       1
 ffffa13ffce7aae0       1
Length: 80, dtype: int64

In [25]:
for val in train['masked'].str.extractall('( (?=[a-fA-F0-9]*[a-fA-F])(?=[a-fA-F0-9]*[0-9])[a-fA-F0-9]{4,}(?: |$))').value_counts().index:
    print(val)

(' BDB0113 ',)
(' BDB1507 ',)
(' BDB0087 ',)
(' 6B9C ',)
(' ffff9ee3fa2ccc90 ',)
(' ffff8c353a2c6c90 ',)
(' ffff8ff6fa2c6a20 ',)
(' ffff8c353a2cb6c0 ',)
(' ffff8ff6fa2cb6c0 ',)
(' ffff95743a2c6c90 ',)
(' ffff95743a2cb4b0 ',)
(' ffff95743a2cb6c0 ',)
(' ffff9cf9ba2cb6c0 ',)
(' ffff98c3fa2d2120 ',)
(' ffff98c3fa2d28a0 ',)
(' ffff9cf9ba2c69c0 ',)
(' ffff9cf9ba2c6a20 ',)
(' ffff95743a2c6e10 ',)
(' ffff9cf9ba2c6ae0 ',)
(' ffff9cf9ba2c6c90 ',)
(' ffff9cf9ba2cb210 ',)
(' ffff947d7ce7e390 ',)
(' ffff98c3fa2cdc90 ',)
(' 01d6f090bba8ef3f ',)
(' ffff9ee3fa2ca210 ',)
(' ffff9ee3fa2cca20 ',)
(' ffff947d7ce7e120 ',)
(' ffff9ee3fa2ccd50 ',)
(' ffff9ee3fa2d1390 ',)
(' ffff9ee3fa2d18a0 ',)
(' ffff9fcb7ce7aa20 ',)
(' ffff9fcb7ce7f4b0 ',)
(' ffff9fcb7ce7f6c0 ',)
(' ffff9fd43a2c69c0 ',)
(' ffff9fd43a2c6c90 ',)
(' ffff9fd43a2cb6c0 ',)
(' ffffa13ffce78210 ',)
(' ffffa13ffce7a9c0 ',)
(' ffffa13ffce7aa20 ',)
(' ffff947d7ce7e2a0 ',)
(' ffff9428a745e8a0 ',)
(' ffff947d7ce79e10 ',)
(' ffff947d7ce79ae0 ',)
(' 3d38

In [26]:
for val in train['masked'].str.extractall('( (?=[a-fA-F0-9]*[a-fA-F])(?=[a-fA-F0-9]*[0-9])[a-fA-F0-9]{4,}(?:\W|$))').value_counts().index:
    print(val)

(' BDB0113 ',)
(' BDB1507 ',)
(' BDB0087 ',)
(' 6B9C ',)
(' ffff95743a2c6c90 ',)
(' ffff8c353a2c6c90 ',)
(' ffff9ee3fa2ccc90 ',)
(' ffff8ff6fa2c6a20 ',)
(' ffff8c353a2cb6c0 ',)
(' ffff8ff6fa2cb6c0 ',)
(' ffff95743a2cb6c0 ',)
(' ffff9575f7c87c30)',)
(' ffff9ee3fa2d1390)',)
(' ffff9ee3fa2d1390 ',)
(' ffff9ee3fa2ccd50 ',)
(' ffff95743a2c6e10 ',)
(' ffff9ee3fa2cca20 ',)
(' ffff95743a2cb210)',)
(' ffff9ee3fa2ca210 ',)
(' ffff9cf9edc87db0)',)
(' ffff95743a2cb2a0)',)
(' ffff9cf9ba2cb6c0 ',)
(' ffff9cf9ba2cb210)',)
(' ffff9cf9ba2cb210 ',)
(' ffff95743a2cb4b0 ',)
(' ffff9cf9ba2c6c90 ',)
(' ffff9cf9ba2c6ae0 ',)
(' ffff9ee3fa2d1420)',)
(' ffff9cf9ba2c69c0 ',)
(' ffff98c3fa2d28a0 ',)
(' ffff98c3fa2d2390)',)
(' ffff98c3fa2d2120 ',)
(' ffff98c3fa2cdc90 ',)
(' ffff9cf9ba2c6a20 ',)
(' 01d6f090bba8ef3f ',)
(' ffff9ee3fa2d18a0 ',)
(' ffff9fd43a2cb4b0)',)
(' ffffffff99451650)',)
(' ffffffff98251650)',)
(' ffffffff92251650)',)
(' ffffffff85651650)',)
(' ffffa13ffce7f390)',)
(' ffffa13ffce7aae0 ',)
(' ffff

In [27]:
train['masked'][4]

'type=SYSCALL msg=audit(1603094402.016:52981): arch=c000003e syscall=2 success=yes exit=3 a0=7ff220e805a4 a1=80000 a2=1 a3=7ff2210864f8 items=1 ppid=5877 pid=5878 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=45 comm="date" exe="/usr/bin/date" subj=system_u:system_r:sysstat_t:s0-s0:c0.c1023 key="audit-wazuh-r" type=CWD msg=audit(1603094402.016:52981):  cwd="/root" type=PATH msg=audit(1603094402.016:52981): item=0 name="/etc/ld.so.cache" inode=69008420 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603094402.016:52981): proctitle=64617465002B2564'

In [28]:
train['masked'] = train['masked'].str.replace('(?<==)(?=[a-fA-F0-9]*[a-fA-F])[a-fA-F0-9]+(?=\W|$)', '<NUM>', regex=True)
train['masked'] = train['masked'].str.replace('(?<=:)(?=[a-fA-F0-9]*[a-fA-F])[a-fA-F0-9]+(?=\s|$)', '<NUM>', regex=True)
train['masked'] = train['masked'].str.replace('(?<=\')(?=[a-fA-F0-9]*[a-fA-F])[a-fA-F0-9]+(?=\')', '<NUM>', regex=True)
train['masked'] = train['masked'].str.replace(
    '(?<= )(?=[a-fA-F0-9]*[a-fA-F])(?=[a-fA-F0-9]*[0-9])[a-fA-F0-9]{4,}(?=\W|$)', '<NUM>', regex=True)

In [29]:
train['masked'][4]

'type=SYSCALL msg=audit(1603094402.016:52981): arch=<NUM> syscall=2 success=yes exit=3 a0=<NUM> a1=80000 a2=1 a3=<NUM> items=1 ppid=5877 pid=5878 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=45 comm="date" exe="/usr/bin/date" subj=system_u:system_r:sysstat_t:s0-s0:c0.c1023 key="audit-wazuh-r" type=CWD msg=audit(1603094402.016:52981):  cwd="/root" type=PATH msg=audit(1603094402.016:52981): item=0 name="/etc/ld.so.cache" inode=69008420 dev=<NUM>:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603094402.016:52981): proctitle=<NUM>'

In [30]:
mixed = train['masked'].str.extractall('(.(?=[a-fA-F0-9]*[a-fA-F])(?=[a-fA-F0-9]*[0-9])[a-fA-F0-9]{4,}(?:\W|$))')
for m in mixed.value_counts().index:
    print(m)

('.c1023 ',)
('Jdbc4:',)
('ha256\\',)
('u001b[',)
('"08da3af15323a51fd9f130fb404a84e1\\',)
('u07FD\\',)
('"a1597e7b4dbf57b83fc3c82473b174f3\\',)
('@6de0bc6b[',)
('u07FB\\',)
('u05F7\\',)
('"f9b2d381198c4acf2d14072db4f455d5\\',)
('"b0d06d534869599439374d6b1888668e\\',)
('{7DEB0}',)
('H1f394A/',)
('@1b2c3c77[',)
('md160.',)
('@db8db2e ',)
("_3cd2fea'",)
('@76dcda74 ',)
('_3cd2fea\\',)
("_4ceeb83'",)
('@46e5f6fb ',)
('@3dade9a2 ',)
('nc051"',)
('@35464fdd ',)
('@2a77f8ea ',)
('@197a8899 ',)
('_60e249c\\',)
('ma1509.',)
('_4ceeb83\\',)
('tdec051"',)
('pdf12"',)
('_81f1a6\\',)
('ha256.',)
('/dc240.',)
('-ab40.',)
('rfc2734"',)
("_81f1a6'",)
('@7cf8b88c ',)
('O6ba1A"',)
('-be7129879836418bb19f6f82eb433387.',)
('-be7129879836418bb19f6f82eb433387"',)
('pdf14"',)
('ha512"',)
('/dc210.',)
('ha512.',)
('"0701c81f7a588b81c051d2c69714792b\\',)
('-20ef8c23518c4ed0be942c220e7c4737.',)
('oc2cddb"',)
('/dc25.',)
('ieee802154/',)
('"6e168ff92f41367671bc57cf2609b891\\',)
("_60e249c'",)
('"3cfc2a204fed140

In [31]:
train[train['masked'].str.contains('<NUM>')]['first_word'].value_counts()

type=SYSCALL    116496
File               363
type=AVC           342
E:                 297
error:             276
oscap:             151
Feb                 76
Jan                 39
Oct                 13
Mar                 11
--MARK--:            3
Nov                  2
Name: first_word, dtype: int64

In [32]:
train[(train['first_word']=='--MARK--:') & train['masked'].str.contains('<NUM>')]

,level,full_log,first_word,masked
id,,,,
205605,0,"--MARK--: N[$9H&R]stqazsx,D)'26MPV?C[E/AeRpf]+...",--MARK--:,"--MARK--: N[$9H&R]stqazsx,D)'26MPV?C[E/AeRpf]+..."
340462,0,--MARK--: Klq)#FqoV_]r_@[C!s.t)E!9xlY^M?^PWIM7...,--MARK--:,--MARK--: Klq)#FqoV_]r_@[C!s.t)E!9xlY^M?^PWIM7...
405156,0,--MARK--: PC8A$Siff!_zTYloi@Mon@;l&'/NL;3OhzX8...,--MARK--:,--MARK--: PC8A$Siff!_zTYloi@Mon@;l&'/NL;3OhzX8...


In [33]:
train[(train['first_word']=='oscap:') & train['masked'].str.contains('<NUM>')]

,level,full_log,first_word,masked
id,,,,
3396,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016030...",oscap:,"oscap: msg: ""xccdf-result"", scan-id: ""00016030..."
3661,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016020...",oscap:,"oscap: msg: ""xccdf-result"", scan-id: ""00016020..."
5695,5,"oscap: msg: ""xccdf-result"", scan-id: ""00016019...",oscap:,"oscap: msg: ""xccdf-result"", scan-id: ""00016019..."
7206,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016052...",oscap:,"oscap: msg: ""xccdf-result"", scan-id: ""00016052..."
10008,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016020...",oscap:,"oscap: msg: ""xccdf-result"", scan-id: ""00016020..."
...,...,...,...,...
451908,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016007...",oscap:,"oscap: msg: ""xccdf-result"", scan-id: ""00016007..."
455193,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016028...",oscap:,"oscap: msg: ""xccdf-result"", scan-id: ""00016028..."
456070,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016023...",oscap:,"oscap: msg: ""xccdf-result"", scan-id: ""00016023..."


In [34]:
for idx in train[(train['first_word']=='--MARK--:') & train['masked'].str.contains('<NUM>')].index:
    print(train['full_log'][idx], '\n')
    print(train['masked'][idx], '\n')

--MARK--: N[$9H&R]stqazsx,D)'26MPV?C[E/AeRpf]+FWTF7ujl&Ke'V9z_4uQWUV&%f;#fvq+F4pV/TQ&P;1N(BzoV3J(Q$+%t,ZOc-+tB)mKsLt^9Yf4lzJY!crBpP,_Y7Sz'ZkBc3ieY7f+aVVQDOvd&7%C!$!X)j6%8Q^!Jko8qbAOq]TM_^RxK^vW[]Bzm2PC3n8gDLKpUg[Q^-WzB-w3;Xttkn!;mgBz4%XMbNh4Mj$blLAUdkzff+OB58MjSbnd7qtP0,!!OUE^*wO[$N8ByFATALdQt.06=J&rB6pGdQkH0)rP#nH6ZSNlmxaSJ37F0e0$V8YnN,-H^AwO8]0sumJptU#[Lk--+Bky%BCn_D_iL[dUrO^P!R3a8YBeGRx*E&hn*]ENy6'O2LlDX3Rx(Z]IDOBRqfSvil(johqn)3^[hVHmB6EyU,hGIJX-Zf]+O)J2jKuq#%mviK8YrRrwE4bbCvX*D5878wPH&7N2x&&.jm8QDw9ng!;,)npJ%qf.WGkM@p#HW-3?uO![Mx;NMfRD@2#&$in)@g(s56EDz&4v/X8Z2'[ai)EX;a!RW[aLcR*h7e8%AL=G8#o+#1Rag?GS5t+[wL2BcVCAL+MW6n_bu148*YPpu0Kbir]YyYaibanwMw%Ts'J*6=c4'h&O=1l)+B,qi'O 

--MARK--: N[$9H&R]stqazsx,D)'26MPV?C[E/AeRpf]+FWTF7ujl&Ke'V9z_4uQWUV&%f;#fvq+F4pV/TQ&P;1N(BzoV3J(Q$+%t,ZOc-+tB)mKsLt^9Yf4lzJY!crBpP,_Y7Sz'ZkBc3ieY7f+aVVQDOvd&7%C!$!X)j6%8Q^!Jko8qbAOq]TM_^RxK^vW[]Bzm2PC3n8gDLKpUg[Q^-WzB-w3;Xttkn!;mgBz4%XMbNh4Mj$blLAUdkzff+OB58MjSbnd7qtP0,!!OUE^*wO[$N8ByFATALdQt.06=J&rB6pGdQkH0)rP#nH6

In [35]:
for idx in train[(train['first_word']=='oscap:') & train['masked'].str.contains('<NUM>')].index:
    print(train['full_log'][idx], '\n')
    print(train['masked'][idx], '\n')

oscap: msg: "xccdf-result", scan-id: "0001603070557", content: "ssg-centos-7-ds.xml", title: "Record Events that Modify the System's Discretionary Access Controls - setxattr", id: "xccdf_org.ssgproject.content_rule_audit_rules_dac_modification_setxattr", result: "fail", severity: "low", description: "At a minimum, the audit system should collect file permission changes for all users and root. If the auditd daemon is configured to use the augenrules program to read audit rules during daemon startup (the default), add the following line to a file with suffix .rules in the directory /etc/audit/rules.d: -a always,exit -F arch=b32 -S setxattr -F auid>=1000 -F auid!=4294967295 -F key=perm_mod If the system is 64 bit then also add the following line: -a always,exit -F arch=b64 -S setxattr -F auid>=1000 -F auid!=4294967295 -F key=perm_mod If the auditd daemon is configured to use the auditctl utility to read audit rules during daemon startup, add the following line to /etc/audit/audit.rules fi

In [36]:
show_contains('<NUM>', col='masked', first_words={'type=AVC'})

type=AVC msg=audit(1613366627.272:4960): avc:  denied  { execute } for  pid=10016 comm="sudo" name="agent_control" dev="dm-0" ino=404839312 scontext=system_u:system_r:httpd_t:s0 tcontext=system_u:object_r:var_t:s0 tclass=file permissive=1 type=AVC msg=audit(1613366627.272:4960): avc:  denied  { read open } for  pid=10016 comm="sudo" path="/var/esild/bin/agent_control" dev="dm-0" ino=404839312 scontext=system_u:system_r:httpd_t:s0 tcontext=system_u:object_r:var_t:s0 tclass=file permissive=1 type=AVC msg=audit(1613366627.272:4960): avc:  denied  { execute_no_trans } for  pid=10016 comm="sudo" path="/var/esild/bin/agent_control" dev="dm-0" ino=404839312 scontext=system_u:system_r:httpd_t:s0 tcontext=system_u:object_r:var_t:s0 tclass=file permissive=1 type=SYSCALL msg=audit(1613366627.272:4960): arch=<NUM> syscall=59 success=yes exit=0 a0=<NUM> a1=<NUM> a2=<NUM> a3=5 items=0 ppid=9939 pid=10016 auid=4294967295 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=429496729

In [37]:
train['masked'] = train['masked'].str.replace('(?<=[^a-zA-Z0-9])(\d+)(?=[^a-zA-Z0-9]|$)', '<NUM>', regex=True)
show_contains('<NUM>', col='masked', cnt=5)

<DATE> <TIME> localhost kibana: {"type":"error","@timestamp":"<TS>","tags":["warning","stats-collection"],"pid":<NUM>,"level":"error","error":{"message":"No Living connections","name":"Error","stack":"Error: No Living connections\n    at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:<NUM>:<NUM>)\n    at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:<NUM>:<NUM>)\n    at process._tickCallback (internal/process/next_tick.js:<NUM>:<NUM>)"},"message":"No Living connections"} 

<DATE> <TIME> localhost logstash: [<TS>,<NUM>][INFO ][logstash.outputs.elasticsearch] retrying failed action with response code: <NUM> ({"type"=>"unavailable_shards_exception", "reason"=>"[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>][<NUM>] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>][<NUM>]] containing [<NUM>] requests]"}) 

<DATE> <TIME> localhost kibana: {"type":"error

## Check

In [38]:
train[train['level']==6]['masked'].value_counts()

<DATE> <TIME> localhost kernel: device virbr0-nic entered promiscuous mode                                                           3
<DATE> <TIME> localhost kernel: device enp2s0 entered promiscuous mode                                                               2
<DATE> <TIME> localhost useradd[<NUM>]: new group: name=test, GID=<NUM>                                                              1
<DATE> <TIME> localhost sshd[<NUM>]: error: maximum authentication attempts exceeded for root from <IP> port <NUM> ssh2 [preauth]    1
<DATE> <TIME> localhost sshd[<NUM>]: Bad protocol version identification '\<NUM>' from <IP> port <NUM>                               1
Name: masked, dtype: int64

In [39]:
train[train['level']==4]['masked'].value_counts()

<DATE> <TIME> sv260 sshd[<NUM>]: Did not receive identification string from <IP>                   5
<DATE> <TIME> localhost sshd[<NUM>]: Did not receive identification string from <IP> port <NUM>    5
Name: masked, dtype: int64

In [40]:
train[train['level']==2]['masked'].value_counts()

The average number of logs between <NUM>:<NUM> and <NUM>:<NUM> is <NUM>. We reached <NUM>.                                                                              11
<DATE> <TIME> localhost sudo:  apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/curl -XGET localhost:<NUM>/_cat/snapshots/esild_backup     1
Name: masked, dtype: int64

## apply masking

In [41]:
TARGET_COLUMN = 'full_log'
PATTERNS = [('\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z?', '<TS>'), 
            # YEAR, MON, DAY, TIME
            ('\d{4}(?= (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))', '<YEAR>'), 
            ('(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}\s+\d{2}:\d{2}:\d{2}', '<DATE> <TIME>'), 
            ('[A-Za-z]{3}\s+<DATE> <TIME>\s+\d{4}', '<DAY> <DATE> <TIME> <YEAR>'), 
            # IP
            ('127.0.0.1', 'localhost'),
            ('\d+\.\d+\.\d+\.\d+(?:\.\d+)?', '<IP>'), 
            # HEX, NUM
            ('(?<![0-9a-fA-F])0x[0-9a-fA-F]+(?=\W|$)', '<HEX>'), 
            ('(?<=\W)(?=[a-fA-F0-9\-]*[0-9])(?=[a-fA-F0-9\-]*[a-fA-F])[a-fA-F0-9]{3,}(?:-[a-fA-F0-9]{3,})+(?=\W|$)', '<SN>'),
            ('(?<==)[a-fA-F0-9]+(?=\W|$)', '<NUM>'),
            ('(?<=:)[a-fA-F0-9]+(?=\s|$)', '<NUM>'),
            ('(?<=\')[a-fA-F0-9]+(?=\')', '<NUM>'),
            ('(?<= )(?=[a-fA-F0-9]*[0-9])[a-fA-F0-9]{4,}(?=\W|$)', '<NUM>'), 
            ('(?<=[^a-zA-Z0-9])(\d+)(?=[^a-zA-Z0-9]|$)', '<NUM>')]

def apply_masking(df):
    for pat, repl in PATTERNS:
        df[TARGET_COLUMN] = df[TARGET_COLUMN].str.replace(pat, repl, regex=True)

In [42]:
train = pd.read_csv('./data/train.csv', index_col=0)
validation = pd.read_csv('./data/validation_sample.csv')
test = pd.read_csv('./data/test.csv', index_col=0)

print(train.shape, validation.shape, test.shape)

(472972, 2) (3, 1) (1418916, 1)


C:\Users\ASIA-11\anaconda3\envs\prj_final\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [43]:
train.head()

,level,full_log
id,,
0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...


In [44]:
validation.head()

,full_log
0,type=ANOM_PROMISCUOUS msg=audit(1600402733.466...
1,"oscap: msg: ""xccdf-result"", scan-id: ""00016007..."
2,Sep 22 10:56:19 localhost kernel: Out of memor...


In [45]:
test.head()

,full_log
id,
1000000,"Feb 8 15:47:26 localhost kibana: {""type"":""err..."
1000001,"Sep 24 03:46:39 localhost kibana: {""type"":""err..."
1000002,type=SYSCALL msg=audit(1611888200.428:210563):...
1000003,"Jan 18 11:24:06 localhost kibana: {""type"":""err..."
1000004,type=SYSCALL msg=audit(1603081202.050:46851): ...


In [46]:
FILES = {'train': train, 'validation': validation, 'test': test}

for file, df in FILES.items():
    apply_masking(df)
    df.to_csv(f'./data/{file}_masked_02.csv')